## Hacker News: when to submit "Ask" or "Show" posts?

I want to analise Hacker News portal where users submit posts. Posts has votes and comments. I will use this dataset: [link](https://www.kaggle.com/hacker-news/hacker-news-posts#HN_posts_year_to_Sep_26_2016.csv). This data set is Hacker News posts from 12 months period (from September 2015).

This data set was preprocessed: it has been reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions. Below are descriptions of the columns:

* `id` - The unique identifier from Hacker News for the post
* `title` - The title of the post
* `url` - The URL that the posts links to, if it the post has a URL
* `num_points` - The number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes
* `num_comments` - The number of comments that were made on the post
* `author` - The username of the person who submitted the post
* `created_at` - The date and time at which the post was submitted

I am specifically interested in posts whose titles begin with either `Ask HN` or `Show HN`. Users submit `Ask HN` posts to ask the Hacker News community a specific question. Likewise, users submit `Show HN` posts to show the Hacker News community a project, product, or just generally something interesting. 

I would like to compare these two types of posts to determine the following:

* Do Ask HN or Show HN receive more comments on average?
* Do posts created at a certain time receive more comments on average?

In [10]:
from csv import reader

opened_file = open("hacker_news.csv")
read_file = reader(opened_file)
hn = list(read_file)

hn_slice = hn[:5]
for row in hn_slice:
    print(row)
    print('\n')

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']


['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']


['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']


['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']




Divide data set into two parts: header and all data.

In [11]:
hn_header = hn[0]
hn = hn[1:]

print(hn_header)
hn_slice = hn[:5]
print('\n')
for row in hn_slice:
    print(row)
    print('\n')

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']


['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']


['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']


['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']


['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']




Since I am only concerned with posts whose titles begin with `Ask HN` or `Show HN`, I'll create new lists of lists containing just the data for those titles.

In [12]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    if title.lower().startswith('ask hn'):
        ask_posts.append(row)
    elif title.lower().startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)
        
print("number of 'ask hn' posts: ", len(ask_posts))
print("number of 'show hn' posts: ", len(show_posts))
print("number of 'other posts: ", len(other_posts))

number of 'ask hn' posts:  1744
number of 'show hn' posts:  1162
number of 'other posts:  17194


Next, let's determine if ask posts or show posts receive more comments on average.

In [14]:
total_ask_comments = 0
for row in ask_posts:
    num_comments = int(row[4])
    total_ask_comments += num_comments
avg_ask_comments = round(total_ask_comments/len(ask_posts))
print("average number of comments in 'ask hn' posts: ", avg_ask_comments)

total_show_comments = 0
for row in show_posts:
    num_comments = int(row[4])
    total_show_comments += num_comments
avg_show_comments = round(total_show_comments/len(show_posts))
print("average number of comments in 'show hn' posts: ", avg_show_comments)

average number of comments in 'ask hn' posts:  14
average number of comments in 'show hn' posts:  10


On average, ask posts receive more comments than show posts. Since ask posts are more likely to receive comments, I'll focus our remaining analysis just on these posts.

Next, I'll determine if ask posts created at a certain time are more likely to attract comments. I'll use the following steps to perform this analysis:

* Calculate the amount of ask posts created in each hour of the day, along with the number of comments received.
* Calculate the average number of comments ask posts receive by hour created.

In [17]:
import datetime as dt

result_list = []
for row in ask_posts:
    created_at = row[6]
    comments_number = int(row[4])
    result_list.append([created_at, comments_number])
    
counts_by_hour = {}
comments_by_hour = {}
for row in result_list:
    date_dt = dt.datetime.strptime(row[0], "%m/%d/%Y %H:%M")
    hour = date_dt.strftime("%H")
    if hour not in counts_by_hour:
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = row[1]
    else:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += row[1]


So, there is two dictionaries:

* `counts_by_hour`: contains the number of ask posts created during each hour of the day
* `comments_by_hour`: contains the corresponding number of comments ask posts created at each hour received

Next, let's use these two dictionaries to calculate the average number of comments for posts created during each hour of the day.

In [23]:
avg_by_hour = []
for hour in counts_by_hour:
    avg_by_hour.append([hour, comments_by_hour[hour]/counts_by_hour[hour]])
display(avg_by_hour)

[['09', 5.5777777777777775],
 ['13', 14.741176470588234],
 ['10', 13.440677966101696],
 ['14', 13.233644859813085],
 ['16', 16.796296296296298],
 ['23', 7.985294117647059],
 ['12', 9.41095890410959],
 ['17', 11.46],
 ['15', 38.5948275862069],
 ['21', 16.009174311926607],
 ['20', 21.525],
 ['02', 23.810344827586206],
 ['18', 13.20183486238532],
 ['03', 7.796296296296297],
 ['05', 10.08695652173913],
 ['19', 10.8],
 ['01', 11.383333333333333],
 ['22', 6.746478873239437],
 ['08', 10.25],
 ['04', 7.170212765957447],
 ['00', 8.127272727272727],
 ['06', 9.022727272727273],
 ['07', 7.852941176470588],
 ['11', 11.051724137931034]]

This format makes it hard to identify the hours with the highest values. Let's finish by sorting the list of lists and printing the five highest values in a format that's easier to read.

In [34]:
swap_avg_by_hour = []
for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])
#print(swap_avg_by_hour)
sorted_swap = sorted(swap_avg_by_hour, reverse = True)
print("Top 5 Hours for Ask Posts Comments:")

for avg, hour in sorted_swap[:5]:
    hour_dt = dt.datetime.strptime(hour, "%H")
    hour_str = hour_dt.strftime("%H:%M")
    output = "{h}: {avg:.2f} average comments per post".format(h=hour_str, avg=avg)
    print(output)

Top 5 Hours for Ask Posts Comments:
15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


In the description of `created_at` column: the date and time the post was made (the time zone is Eastern Time in the US). To collect as many comments as posiible posts should be submitted at 15:00, 02:00, 20:00, 16:00 or 21:00 Estern Time in US.